In [46]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load prediction files
text_df = pd.read_excel("/content/text_probabilities.xlsx")       # Replace with actual file name
audio_df = pd.read_excel("/content/audio_test_predictions_final_final.xlsx")     # Replace with actual file name
face_df = pd.read_csv("/content/Unseen Predictions with Uncertainty.csv")          # Replace with actual file name
labels_df = pd.read_csv("/content/Detailed Labels.csv")      # Replace with actual file name

# Merge on common Participant_IDs only (inner join)
merged_df = text_df.merge(text_df, on='Participant_ID', how='inner')
merged_df = text_df.merge(audio_df, on='Participant_ID', how='inner')
merged_df = merged_df.merge(face_df, on='Participant_ID', how='inner')
merged_df = merged_df.merge(labels_df[['Participant_ID', 'Depression_label']], on='Participant_ID', how='inner')

# Drop any rows with missing values just in case
merged_df = merged_df.dropna()

# print(merged_df.head())

# Compute average probabilities
merged_df['avg_prob_0'] = merged_df[['Text_Prob_0', 'audio_prob_0', 'face_prob_0']].mean(axis=1)
merged_df['avg_prob_1'] = merged_df[['Text_Prob_1', 'audio_prob_1', 'face_prob_1']].mean(axis=1)

# Predict based on which probability is higher
merged_df['predicted'] = (merged_df['avg_prob_1'] > merged_df['avg_prob_0']).astype(int)


# True labels and predictions
y_true = merged_df['Depression_label']
y_pred = merged_df['predicted']

# Evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Accuracy: 0.7000

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.76      0.78        34
           1       0.53      0.56      0.55        16

    accuracy                           0.70        50
   macro avg       0.66      0.66      0.66        50
weighted avg       0.71      0.70      0.70        50


Confusion Matrix:
[[26  8]
 [ 7  9]]
